In [ ]:
# pip install -r requirements.txt
# python -m spacy download en_core_web_md
from transformers import pipeline

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis",model="cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
import re
import spacy
import pandas as pd
from transformers import pipeline
from pathlib import Path
import hashlib

In [ ]:
# Create folders
Path("data/raw").mkdir(parents=True, exist_ok=True)
Path("data/processed").mkdir(parents=True, exist_ok=True)

In [ ]:
# Load data
df = pd.read_csv("data/raw/renewable_redditcomments.csv")
df = df.dropna(subset=["comment_body"]).copy()

In [ ]:
nlp = spacy.load("en_core_web_md")

In [ ]:
def text_preprocessing(text):
  text = re.sub(r'\s+', ' ', text).strip()
  doc = nlp(text)
  cleaned_text = " ".join([token.text for token in doc if not token.is_stop and not token.is_punct and not token.like_url and not token.like_email])
  normalized_text = cleaned_text.lower()
  return normalized_text

df["clean_text"] = df["comment_body"].apply(text_preprocessing)

In [ ]:
def get_sentiment(text):
  result = sentiment_pipeline(text)
  score = result[0]['score']
  sentiment = result[0]['label']
  return pd.Series([sentiment, score], index=['sentiment_label', 'sentiment_score_r'])

df[['sentiment_label', 'sentiment_score_r']] = df['clean_text'].apply(get_sentiment)

In [ ]:
df.drop('clean_text', axis='columns', inplace=True)

In [ ]:
df.to_csv('data/processed/Sentiment_Analysis_Roberta.csv', index=False)